In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk
import os
import numpy as np

In [ ]:
def mostrar_imagem_video(file_path, modo, root):
    global imagem_carregada, imagem_original, canvas, img_container, blur_var, sharpen_var, emboss_var, laplacian_var, canny_var, sobel_var, cores_var, cinza_var, binario_var
    cap = cv2.VideoCapture(file_path)
    
    if not cap.isOpened():
        print("Erro: não foi possível abrir o arquivo de vídeo.")
        return
    
    # Configurações da segunda janela
    segunda_janela = tk.Toplevel(root)
    segunda_janela.title("Manipulação de Imagens e Vídeos")
    segunda_janela.attributes('-fullscreen', True)
    
    # PanedWindow para o painel superior
    paned_window_top = tk.PanedWindow(segunda_janela, orient=tk.HORIZONTAL, bd=4, relief='raised', bg='#e6f2ff')
    paned_window_top.pack(side=tk.TOP, fill=tk.X, pady=20)

    # Frame para a label e checkboxes
    frame_top = tk.Frame(paned_window_top, bg="#e6f2ff")
    paned_window_top.add(frame_top)

    # Label "Aplicar Filtro"
    label_filtro = tk.Label(frame_top, text="Aplicar Filtro", font=("Arial", 24), bg="#e6f2ff")
    label_filtro.pack(side=tk.LEFT, padx=20)

    # Variáveis para armazenar os estados das checkboxes
    blur_var = tk.IntVar()
    sharpen_var = tk.IntVar()
    emboss_var = tk.IntVar()
    laplacian_var = tk.IntVar()
    canny_var = tk.IntVar()
    sobel_var = tk.IntVar()

    # Checkboxes
    chk_blur = tk.Checkbutton(frame_top, text="Blur", variable=blur_var, bg="#e6f2ff", font=("Arial", 14))
    chk_blur.pack(side=tk.LEFT, padx=10)

    chk_sharpen = tk.Checkbutton(frame_top, text="Sharpen", variable=sharpen_var, bg="#e6f2ff", font=("Arial", 14))
    chk_sharpen.pack(side=tk.LEFT, padx=10)

    chk_emboss = tk.Checkbutton(frame_top, text="Emboss", variable=emboss_var, bg="#e6f2ff", font=("Arial", 14))
    chk_emboss.pack(side=tk.LEFT, padx=10)

    chk_laplacian = tk.Checkbutton(frame_top, text="Laplacian", variable=laplacian_var, bg="#e6f2ff", font=("Arial", 14))
    chk_laplacian.pack(side=tk.LEFT, padx=10)

    chk_canny = tk.Checkbutton(frame_top, text="Canny", variable=canny_var, bg="#e6f2ff", font=("Arial", 14))
    chk_canny.pack(side=tk.LEFT, padx=10)

    chk_sobel = tk.Checkbutton(frame_top, text="Sobel", variable=sobel_var, bg="#e6f2ff", font=("Arial", 14))
    chk_sobel.pack(side=tk.LEFT, padx=10)

    # Label "Tons de Imagem"
    label_tons_imagem = tk.Label(frame_top, text="Tons de Imagem", font=("Arial", 24), bg="#e6f2ff")
    label_tons_imagem.pack(side=tk.LEFT, padx=20)

    # Variáveis para os estados das checkboxes de tons de imagem
    cores_var = tk.IntVar()
    cinza_var = tk.IntVar()
    binario_var = tk.IntVar()

    # Checkboxes de tons de imagem
    chk_cores = tk.Checkbutton(frame_top, text="Cores", variable=cores_var, bg="#e6f2ff", font=("Arial", 14))
    chk_cores.pack(side=tk.LEFT, padx=10)

    chk_cinza = tk.Checkbutton(frame_top, text="Cinza", variable=cinza_var, bg="#e6f2ff", font=("Arial", 14))
    chk_cinza.pack(side=tk.LEFT, padx=10)

    chk_binario = tk.Checkbutton(frame_top, text="Binário", variable=binario_var, bg="#e6f2ff", font=("Arial", 14))
    chk_binario.pack(side=tk.LEFT, padx=10)

    # Canvas para exibir a imagem
    canvas = tk.Canvas(segunda_janela, bg="white")
    canvas.pack(fill=tk.BOTH, expand=True)

    # PanedWindow para os botões na parte inferior
    paned_window_bottom = tk.PanedWindow(segunda_janela, orient=tk.HORIZONTAL, bd=4, relief='raised', bg='#e6f2ff')
    paned_window_bottom.pack(side=tk.BOTTOM, fill=tk.X, pady=20)

    # Frame para os botões
    frame_botoes = tk.Frame(paned_window_bottom, bg="#e6f2ff")
    paned_window_bottom.add(frame_botoes)

    # Função para posicionar botões proporcionalmente
    def pack_proportionally(widget, side, padx=10):
        widget.pack(side=side, padx=padx, expand=True)
    
    # Botão "Voltar"
    btn_voltar = tk.Button(frame_botoes, text="Voltar", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=segunda_janela.destroy)
    pack_proportionally(btn_voltar, tk.LEFT)

    # Botão "Zoom+"
    btn_zoom_mais = tk.Button(frame_botoes, text="Zoom+", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=lambda: print("Zoom In"))
    pack_proportionally(btn_zoom_mais, tk.LEFT)

    # Botão "Zoom-"
    btn_zoom_menos = tk.Button(frame_botoes, text="Zoom-", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=lambda: print("Zoom Out"))
    pack_proportionally(btn_zoom_menos, tk.LEFT)

    # Botão "Recorte"
    btn_recorte = tk.Button(frame_botoes, text="Recorte", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=lambda: print("Recortar"))
    pack_proportionally(btn_recorte, tk.LEFT)

    # Botão "Desfazer Recorte"
    btn_desfazer_recorte = tk.Button(frame_botoes, text="Desfazer Recorte", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=lambda: print("Desfazer Recorte"))
    pack_proportionally(btn_desfazer_recorte, tk.LEFT)

    # Botão "Salvar"
    btn_salvar = tk.Button(frame_botoes, text="Salvar", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=lambda: print("Salvar"))
    pack_proportionally(btn_salvar, tk.LEFT)

    # Botão "Sair"
    btn_sair = tk.Button(frame_botoes, text="Sair", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=segunda_janela.destroy)
    pack_proportionally(btn_sair, tk.RIGHT)

    def mostra_quadro():
        if not cap.isOpened():
            print("Erro ao capturar o quadro.")
            return

        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            # Obter a largura e altura da segunda janela
            screen_width = segunda_janela.winfo_screenwidth()
            screen_height = segunda_janela.winfo_screenheight()
            # Calcular o tamanho da imagem
            img_width = int(screen_width * 0.3)
            img_height = int(screen_height * 0.3)
            # Redimensionar a imagem
            img = img.resize((img_width, img_height), Image.LANCZOS)
            imgtk = ImageTk.PhotoImage(image=img)
            # Centralizar a imagem no canvas, com ajuste no eixo y
            x_centro = (screen_width - img_width) // 2
            y_centro = (screen_height - img_height) // 4  # Ajuste no eixo y para subir a imagem
            # Exibir a imagem no canvas
            img_container = canvas.create_image(x_centro, y_centro, anchor=tk.NW, image=imgtk)
            canvas.imgtk = imgtk  # Manter a referência para evitar a coleta de lixo
        else:
            print("Erro ao capturar o quadro.")
        
        if segunda_janela.winfo_exists():
            canvas.after(10, mostra_quadro)
        else:
            cap.release()
    
    mostra_quadro()

    # Exibir as coordenadas x e y dos botões
    def print_coordinates(event=None):
        print(f"Botão 'Sair' - x: {btn_sair.winfo_x()}, y: {btn_sair.winfo_y()}")
        print(f"Botão 'Voltar' - x: {btn_voltar.winfo_x()}, y: {btn_voltar.winfo_y()}")

    # Inicializar o evento para exibir as coordenadas dos botões
    segunda_janela.bind("<Button-2>", print_coordinates)

In [ ]:
def selecionar_arq(modo, root):
    # Detectar sistema operacional e definir diretório inicial
    inicial_dir = os.path.expanduser("/home/wallace/Documentos/TETI/teti_imagens/")  # Define o diretório inicial como o diretório principal do usuário
    file_path = filedialog.askopenfilename(initialdir=inicial_dir, filetypes=[("All Files", "*.*"), ("Image files", "*.jpg *.png"), ("Video files", "*.mp4 *.avi")])
    if file_path:
        mostrar_imagem_video(file_path, modo, root)

In [ ]:
def main():
    root = tk.Tk()
    root.title("Manipulação de Imagens e Vídeos")
    root.geometry("1200x900")
    
    # Estilo da janela principal
    root.configure(background="#e6f2ff")

    # Estilo dos componentes
    style = ttk.Style()
    style.configure("TLabel", background="#e6f2ff", foreground="#333333", padding=(15, 10, 15, 10), font=("Arial", 14))
    style.configure("TCombobox", foreground="#333333", padding=(15, 10, 20, 10), font=("Arial", 14))

    # Dropdown para selecionar o tipo de arquivo
    tipo_arquivo_label = ttk.Label(root, text="Selecione o Tipo de Arquivo", style="TLabel")
    tipo_arquivo_label.place(x=400, y=100)
    
    tipo_arquivo = ttk.Combobox(root, values=["Imagem", "Vídeo"], style="TCombobox")
    tipo_arquivo.place(x=600, y=100)

    # Dropdown para selecionar o modo
    modo_label = ttk.Label(root, text="Selecione o Modo", style="TLabel")
    modo_label.place(x=400, y=200)
    
    modo = ttk.Combobox(root, values=["Independente", "Cascata"], style="TCombobox")
    modo.place(x=600, y=200)

    # Função para associar dropdowns
    def modo_selecionado(event):
        modo_selecionado = modo.get()
        if tipo_arquivo.get():
            selecionar_arq(modo_selecionado, root)

    # Associe o evento de mudança de seleção com a função
    modo.bind("<<ComboboxSelected>>", modo_selecionado)

    # Botões Avançar e Sair com tkinter Button
    btn_avancar = tk.Button(root, text="Avançar", font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20, command=lambda: selecionar_arq(modo.get(), root))
    btn_avancar.place(x=100, y=700)

    btn_sair = tk.Button(root, text="Sair", command=root.destroy, font=("Arial", 14), bg="#007acc", fg="#ffffff", padx=30, pady=20)
    btn_sair.place(x=980, y=700)

    root.mainloop()

In [ ]:
if __name__ == "__main__":
    main()